In [1]:
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd
import html5lib 
from lxml import html
import numpy as np
from helpers import *
#import helpers as helpers

In [2]:
# The URL of the "start" of the IS-Academia page listing the students.
home_url = "http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.filter?ww_x_GPS=-1&ww_i_reportModel=133685247"

# The general form of the URL, with the fields to be replaced later (e.g. [UNITE_ACADEMIQUE])
base_url = "http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.html?ww_x_GPS=-1&ww_i_reportModel=133685247&ww_i_reportModelXsl=133685270&ww_x_UNITE_ACAD=[UNITE_ACADEMIQUE]&ww_x_PERIODE_ACAD=[PERIODE_ACADEMIQUE]&ww_x_PERIODE_PEDAGO=[PERIODE_PEDAGOGIQUE]&ww_x_HIVERETE=null"

# Get the raw content from the page
with urllib.request.urlopen(home_url) as url:
    s = url.read()

soup = BeautifulSoup(s, 'html.parser')

# Parse the content
url_section = 0
url_years = {}
url_master = {}

#For loop on all the possible fields with <option> to get the value associated with each year, or each "Bachelor" semester
for link in soup.find_all('option'):
    if link.text == 'Informatique':
        url_section = link.get('value')
    if "20" in link.text:
        #print(link.text)
        url_years[link.text] = link.get('value')
    if 'Master' in link.text or 'Mineur' in link.text:
        url_master[link.text] = link.get('value')

        
print(url_years)
print(url_master)
#frame= frame.drop([2,3,4,5,6,8,9,11],axis=1)
#frame.head(10)

{'2007-2008': '978181', '2011-2012': '123455150', '2016-2017': '355925344', '2012-2013': '123456101', '2014-2015': '213637922', '2009-2010': '978195', '2013-2014': '213637754', '2010-2011': '39486325', '2008-2009': '978187', '2015-2016': '213638028'}
{'Projet Master automne': '249127', 'Master semestre 2': '942192', 'Mineur semestre 2': '2335676', 'Master semestre 1': '2230106', 'Projet Master printemps': '3781783', 'Master semestre 3': '2230128', 'Master semestre 4': '2230140', 'Mineur semestre 1': '2335667'}


In [3]:
def getFullUrl(PeriodeAcad, PeriodePedag):
    url = base_url
    url = url.replace('[UNITE_ACADEMIQUE]', str(url_section))
    url = url.replace('[PERIODE_ACADEMIQUE]', str(url_years[PeriodeAcad]))
    url = url.replace('[PERIODE_PEDAGOGIQUE]', str(url_master[PeriodePedag]))
    return url

Assumptions : 
- All students start in MS1 and do it only once since afterward it will be MS3.

In [4]:
def MasterStart(year_string,master_student,pedag) : 
    current_url = getFullUrl(year_string, pedag)
    with urllib.request.urlopen(current_url) as url:
        html = url.read()
    soup = BeautifulSoup(html, 'html.parser')
    master_frame = pd.read_html(soup.prettify(), header=1)
    if  len(master_frame)>0 :
        master_frame = master_frame[0].drop(0,axis=0)
        master_frame_sciper = master_frame[['No Sciper']]
        master_frame = master_frame.set_index('No Sciper')
        for sciper in master_frame_sciper.values:
            sciper = sciper[0]
            
            if pedag == 'Master semestre 1' or sciper not in master_student.index:
             master_student.loc[sciper] = 0   
             master_student.loc[sciper,pedag] = 1
            else :
             master_student.loc[sciper,pedag] += 1   
            master_student.loc[sciper,'Civilité'] = master_frame.loc[sciper,'Civilité']
            master_student.loc[sciper,'Nom Prénom'] = master_frame.loc[sciper,'Nom Prénom']
            master_student.loc[sciper,'Spécialisation'] = master_frame.loc[sciper,'Spécialisation']
            master_student.loc[sciper,'Mineur'] = master_frame.loc[sciper,'Mineur']
    return master_student

In [5]:
master_student = pd.DataFrame(columns = ['Civilité', 'Nom Prénom', 
                                         'Master semestre 1', 'Master semestre 2', 'Master semestre 3',
                                         'Projet Master printemps', 'Projet Master automne', 
                                         'Spécialisation', 'Mineur'])
master_student.index.name = 'No Sciper'

pedagogic_year = ['Master semestre 1','Master semestre 2','Master semestre 3',
                  'Projet Master printemps','Projet Master automne']
for year in range(2007,2016):
    year_string = str(year) + '-' + str(year+1)
    for pedag in  pedagogic_year :
     master_student = MasterStart(year_string,master_student,pedag)


.Assumptions  : 
 - possible to have project master in spring and autumn
 - if no project master register assume thta student had project outside epfl and add +1 semester

In [6]:
print(master_student.index.is_unique)
## sum over all the master 1,2,3 plus additional master project.
time = (master_student['Master semestre 1']+master_student['Master semestre 2']+
        master_student['Master semestre 3']+master_student['Projet Master printemps']+
        master_student['Projet Master automne']+
        1-(master_student['Projet Master printemps']+ master_student['Projet Master automne'])%1)

master_student['Time to Finish']= time
master_student.head()

True


,Civilité,Nom Prénom,Master semestre 1,Master semestre 2,Master semestre 3,Projet Master printemps,Projet Master automne,Spécialisation,Mineur,Time to Finish
No Sciper,,,,,,,,,,
180027,Madame,Agarwal Megha,1.0,1.0,0.0,0.0,0.0,NaN,NaN,3.0
152232,Monsieur,Anagnostaras David,1.0,1.0,1.0,0.0,0.0,NaN,"Mineur en Management, technologie et entrepren...",4.0
177395,Monsieur,Auroux Damien,1.0,1.0,1.0,0.0,0.0,Internet computing,NaN,4.0
161970,Monsieur,Awalebo Joseph,1.0,1.0,1.0,0.0,0.0,NaN,NaN,4.0
166258,Monsieur,Balet Ken,1.0,1.0,1.0,0.0,0.0,Internet computing,NaN,4.0


In [91]:
import scipy.stats as stats
specialisation = master_student[master_student['Spécialisation']==master_student['Spécialisation']]                        
mean_time_specialisation = specialisation['Time to Finish']
mean_time  = master_student['Time to Finish'].mean()
print(mean_time)
print(mean_time_specialisation.mean())
stats.ttest_1samp(a= mean_time_specialisation, 
                 popmean= mean_time)

3.79435957697
4.20553359684


Ttest_1sampResult(statistic=5.3878631523425309, pvalue=1.6361493733246634e-07)

In [103]:
master_student.describe()

,Master semestre 1,Master semestre 2,Master semestre 3,Projet Master printemps,Projet Master automne,Time to Finish,No Sciper
count,851.000000,851.000000,851.000000,851.000000,851.000000,851.000000,851.000000
mean,0.897767,1.151586,0.681551,0.041128,0.022327,3.794360,199134.712103
std,0.303132,0.583544,0.580771,0.198703,0.147831,1.106671,28550.543931
min,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,128911.000000
25%,1.000000,1.000000,0.000000,0.000000,0.000000,3.000000,177397.000000
50%,1.000000,1.000000,1.000000,0.000000,0.000000,4.000000,199060.000000
75%,1.000000,1.000000,1.000000,0.000000,0.000000,5.000000,221179.000000
max,1.000000,3.000000,3.000000,1.000000,1.000000,8.000000,261146.000000


In [108]:
master_student_grouped = master_student.groupby(master_student.Spécialisation)
mean_spec = pd.DataFrame(columns=['Mean'])

for spec,student in master_student_grouped:
    #print('spec', spec)
    #print('student', student['Time to Finish'])
    mean_spec.loc[spec] = np.mean(student['Time to Finish'].values)

mean_spec

,Mean
Biocomputing,3.181818
Computer Engineering - SP,4.277778
Computer Science Theory,3.000000
Data Analytics,2.800000
Foundations of Software,4.492063
Information Security - SP,3.333333
Internet Information Systems,3.000000
Internet computing,4.272727
Service science,4.333333
"Signals, Images and Interfaces",4.400000


In [63]:
#print(master_student.head(20))
#master_student = master_student.drop('Mineur', axis=1)

master_student_with_Spec = master_student[master_student.Spécialisation == master_student.Spécialisation]
list_Spec = master_student_with_Spec['Spécialisation'].drop_duplicates().values

#print(master_student_with_Spec.shape)
#print(master_student_with_Spec.sort_index().head())

#baseball_newind.sort_index(axis=1).head()
master_student_with_Spec['No Sciper'] = master_student_with_Spec.index
master_student_with_Spec = master_student_with_Spec.set_index(['Spécialisation'])
master_student_with_Spec = master_student_with_Spec.sort_index()
master_student_with_Spec = master_student_with_Spec.set_index([master_student_with_Spec.index, 'No Sciper'])

master_student_with_Spec.head(100)

#print(master_student_with_Spec.head())


/home/paul/anaconda2/lib/python3.5/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Civilité  \
Spécialisation            No Sciper             
Biocomputing              160831     Monsieur   
                          172500       Madame   
                          154573       Madame   
                          183185       Madame   
                          172503     Monsieur   
                          161090     Monsieur   
                          160150     Monsieur   
                          153640     Monsieur   
                          154936     Monsieur   
                          169734     Monsieur   
                          154969     Monsieur   
Computer Engineering - SP 185458     Monsieur   
                          224627     Monsieur   
                          166262     Monsieur   
                          178271     Monsieur   
                          191354     Monsieur   
                          211672       Madame   
                          216690     Monsieur   
                          217301       Madame   
                          214225     Monsieur   
                          185005       Madame   
                          218353     Monsieur   
                          178283     Monsieur   
                          201034     Monsieur   
                          211478     Monsieur   
                          185480     Monsieur   
                          223825     Monsieur   
                          183984     Monsieur   
                          244206     Monsieur   
Computer Science Theory   227443     Monsieur   
...                                       ...   
Foundations of Software   211388     Monsieur   
                          205008       Madame   
                          248065     Monsieur   
                          153882     Monsieur   
                          160915     Monsieur   
                          186060     Monsieur   
                          227942     Monsieur   
                          166264     Monsieur   
                          166263     Monsieur   
                          225743     Monsieur   
                          161794     Monsieur   
                          166935     Monsieur   
                          166124     Monsieur   
                          166229     Monsieur   
                          165819     Monsieur   
                          177399     Monsieur   
                          161357     Monsieur   
                          160718     Monsieur   
                          226638     Monsieur   
                          212654     Monsieur   
                          193046     Monsieur   
                          196065     Monsieur   
                          166746     Monsieur   
                          183895     Monsieur   
                          170823     Monsieur   
                          161402     Monsieur   
                          217469     Monsieur   
                          217629     Monsieur   
Information Security - SP 226826     Monsieur   
                          224331     Monsieur   

                                                           Nom Prénom  \
Spécialisation            No Sciper                                     
Biocomputing              160831                       Dubout Charles   
                          172500            Diatchka Kremena Sotirova   
                          154573                   Benabdallah Zeineb   
                          183185                       Yurovsky Alisa   
                          172503                      Evans Nathaniel   
                          161090                  Habersaat Alexandre   
                          160150                         Lépine Simon   
                          153640            Matthey-de-l'Endroit Loïc   
                          154936                         Resin Pascal   
                          169734                       Koller Yannick   
                          154969                       Ruffieux Simon   


In [90]:
#segments.groupby(segments.name).seg_length.sum().sort_values(ascending=False, inplace=False).head(20)

master_student_with_Spec.groupby(master_student_with_Spec.index).sum()

,Master semestre 1,Master semestre 2,Master semestre 3,Projet Master printemps,Projet Master automne,Time to Finish
"(Biocomputing, 153640)",0.0,0.0,1.0,0.0,0.0,2.0
"(Biocomputing, 154573)",1.0,2.0,2.0,1.0,0.0,7.0
"(Biocomputing, 154936)",0.0,0.0,1.0,0.0,0.0,2.0
"(Biocomputing, 154969)",0.0,0.0,1.0,0.0,0.0,2.0
"(Biocomputing, 160150)",0.0,0.0,1.0,1.0,0.0,3.0
"(Biocomputing, 160831)",0.0,0.0,1.0,0.0,0.0,2.0
"(Biocomputing, 161090)",0.0,0.0,1.0,0.0,0.0,2.0
"(Biocomputing, 169734)",1.0,2.0,2.0,0.0,0.0,6.0
"(Biocomputing, 172500)",0.0,1.0,1.0,0.0,0.0,3.0
"(Biocomputing, 172503)",0.0,0.0,1.0,0.0,0.0,2.0


In [68]:
#master_student['Spécialisation'].drop_duplicates()
print(list_Spec)
master_student_with_Spec.loc[list_Spec[1]]

['Internet computing' 'Biocomputing' 'Foundations of Software'
 'Signals, Images and Interfaces' 'Computer Engineering - SP'
 'Service science' 'Software Systems' 'Information Security - SP'
 'Internet Information Systems' 'Data Analytics' 'Computer Science Theory']


,Civilité,Nom Prénom,Master semestre 1,Master semestre 2,Master semestre 3,Projet Master printemps,Projet Master automne,Time to Finish
No Sciper,,,,,,,,
160831,Monsieur,Dubout Charles,0.0,0.0,1.0,0.0,0.0,2.0
172500,Madame,Diatchka Kremena Sotirova,0.0,1.0,1.0,0.0,0.0,3.0
154573,Madame,Benabdallah Zeineb,1.0,2.0,2.0,1.0,0.0,7.0
183185,Madame,Yurovsky Alisa,1.0,1.0,1.0,0.0,0.0,4.0
172503,Monsieur,Evans Nathaniel,0.0,0.0,1.0,0.0,0.0,2.0
161090,Monsieur,Habersaat Alexandre,0.0,0.0,1.0,0.0,0.0,2.0
160150,Monsieur,Lépine Simon,0.0,0.0,1.0,1.0,0.0,3.0
153640,Monsieur,Matthey-de-l'Endroit Loïc,0.0,0.0,1.0,0.0,0.0,2.0
154936,Monsieur,Resin Pascal,0.0,0.0,1.0,0.0,0.0,2.0


In [85]:

for spec in list_Spec:
    master_student_in_spec = master_student_with_Spec.loc[spec]
    print('Specialisation ' + spec)
    print(master_student_in_spec.loc[:,'Time to Finish'].shape)
    print(stats.ttest_1samp(a= np.array(master_student_in_spec.loc[:,'Time to Finish']), popmean=mean_time))
    print()
    
# Note : we get a NaN pvalue when there is only 1 student in a Specialisation as there is not enough data to perform
# a statistical test.

Specialisation Internet computing
(99,)
Ttest_1sampResult(statistic=3.9315180893794182, pvalue=0.00015749288506068331)

Specialisation Biocomputing
(11,)
Ttest_1sampResult(statistic=-1.1421904023976877, pvalue=0.27998560316699256)

Specialisation Foundations of Software
(63,)
Ttest_1sampResult(statistic=5.377773502058866, pvalue=1.2157140931386113e-06)

Specialisation Signals, Images and Interfaces
(30,)
Ttest_1sampResult(statistic=2.179163639766438, pvalue=0.03758775150391562)

Specialisation Computer Engineering - SP
(18,)
Ttest_1sampResult(statistic=2.7277019357081698, pvalue=0.014321422998079402)

Specialisation Service science
(3,)
Ttest_1sampResult(statistic=1.6169212690951806, pvalue=0.24728661980012823)

Specialisation Software Systems
(16,)
Ttest_1sampResult(statistic=0.086987086915479517, pvalue=0.93183238020497206)

Specialisation Information Security - SP
(6,)
Ttest_1sampResult(statistic=-1.3830787309048176, pvalue=0.22521574404135081)

Specialisation Internet Information S

/home/paul/anaconda2/lib/python3.5/site-packages/numpy/core/_methods.py:82: RuntimeWarning: Degrees of freedom <= 0 for slice
  warnings.warn("Degrees of freedom <= 0 for slice", RuntimeWarning)


We observe that there is a statistically significant difference of Master duration for students choosing Specifications **Internet Computing**, **Foundations of Software**, **Signals, Images and Interfaces**, **Computer Engineering - SP**, **Computer Engineering - SP** and **Data Analytics**